# Clase práctica (90 min) — Limpieza y análisis en **Google Sheets** con dataset de Taylor Swift

**Clase teórica (sin ejecución de código).**  
**Fecha:** 2025-08-18  
**Objetivo general:** Aprender a **limpiar datos heterogéneos** y construir indicadores en *Google Sheets* usando un flujo reproducible y auditable (con `change_log`).  
**Dataset:** `taylor_swift_da_intro.xlsx` (hojas: `raw_songs`, `clean_songs`, `lookup_albums`, `change_log`).

<div style="text-align: center">
    <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/images/w2s1.jpg" width="500">
</div>

---

## Introducción
En esta sesión aprenderás a **detectar y corregir problemas comunes de calidad de datos** y a **transformar columnas** para preparar la información antes del análisis. Luego, convertirás los datos limpios en **métricas y visualizaciones** claras para comunicar hallazgos.

Usaremos el archivo de Taylor Swift como hilo conductor. La práctica operativa será en Google Sheets (no dentro de este notebook).

## Agenda (90 min)
1. (10’) Setup y reglas (no tocar `raw_*`, registrar cambios en `change_log`).
2. (25’) Normalización básica: álbum, fechas, booleanos y numéricos.
3. (20’) Duración a segundos y *split* de autores (transformaciones de texto).
4. (15’) Duplicados y llaves de unicidad.
5. (10’) KPIs y resumen (con y sin tabla dinámica).
6. (10’) Gráficos rápidos y checklist final.
7. (≈5’) **Resumen de aprendizaje y mini evaluación**.

---

## 0) Setup inicial

**Paso 1 — Subir archivo a Google Sheets**
- Descarga `taylor_swift_da_intro.xlsx` y súbelo a **Google Drive**. Ábrelo con **Google Sheets**.

**Paso 2 — Hoja de trabajo**
- Duplica `raw_songs` → nómbrala `clean_songs`. **Nunca** edites `raw_songs`.
- Congela la fila 1, ajusta anchos, activa filtro.

**Paso 3 — Bitácora de cambios**
- Abre `change_log` y agrega tu primera fila:
  - `timestamp`: `=NOW()`  
  - `user`: tu nombre  
  - `sheet`: "clean_songs"  
  - `action`: "create"  
  - `details`: "Duplicada desde raw_songs"  
- Registra **toda modificación estructural** (columnas nuevas, deduplicación, decisiones).

**Buenas prácticas**: usa colores suaves para nuevas columnas, y comentarios en encabezados para documentar (Insertar → Nota).

---

## 0.5) Atajo muy útil: **Buscar y reemplazar** en Google Sheets

Cuando estás empezando, **Buscar y reemplazar** te ahorra muchas fórmulas.

**Cómo abrirlo:** `Ctrl + H` (Windows) / `⌘ + Shift + H` (Mac)

**Reglas para usarlo bien:**
1. **Selecciona primero** la columna o rango donde quieres aplicar el cambio (para no dañar toda la hoja).
2. En la ventana, usa **Buscar** y **Reemplazar con**.
3. Mantén desactivado “Buscar usando expresiones regulares” (no usaremos regex).

**Ejemplos que usaremos hoy:**
- **Espacios dobles**: Buscar `␠␠` (dos espacios) → Reemplazar con `␠` (un espacio). Repite si es necesario.
- **Duración tipo 3m51s**: Buscar `m` → Reemplazar con `:`; luego Buscar `s` → Reemplazar con *(vacío)*.
- **Fechas con guiones o puntos**: Buscar `-` → Reemplazar con `/`; Buscar `.` → Reemplazar con `/`.
- **Números con separadores**: Buscar `_` → Reemplazar con *(vacío)* (si ves valores como `1_000_000`).

> Buen criterio: primero **arregla el texto con Buscar/Reemplazar**, después aplica fórmulas simples.


---
## 1) Normalización básica (álbum, fechas, booleanos, numéricos)

> Regla general: escribe la fórmula en la fila 2 y luego **arrastra/copia hacia abajo**.

### 1.1 Normalizar nombre de álbum (`album_norm`)
**Objetivo:** que el mismo álbum no aparezca con variantes como `FEARLESS`, `Speak now `, `EverMore`, etc.

**Paso 1 — Crea una columna nueva:** `album_norm` (al lado de `album`).

**Fórmula (en la fila 2):**
```gs
=PROPER(TRIM(B2))
```

**Qué significa cada función:**
- `TRIM(B2)` quita espacios “sobrantes” al inicio y al final.
- `PROPER(...)` convierte a formato “Nombre Propio” (mayúscula inicial por palabra).

**Paso 2 — Corrige variantes puntuales con Buscar y reemplazar (Ctrl+H)**
Ejemplos típicos:
- Buscar `Mid nights` → Reemplazar con `Midnights`
- Buscar `Ever More` → Reemplazar con `Evermore`

> Esto es intencional: para principiantes es mejor corregir 2–5 casos manualmente que usar fórmulas complejas.

---

### 1.2 Fechas a formato ISO `yyyy-mm-dd` (`release_iso`)
**Problema:** algunas fechas vienen como `2008-11-11`, `11/11/2008`, `11-11-2008`, etc.

**Opción recomendada (más simple):**
1. Asegúrate de que Google Sheets reconozca la columna como **Fecha** (no como texto).
2. Ve a **Formato → Número → Fecha y hora personalizada**.
3. Elige `yyyy-mm-dd`.

**Si tu columna está como texto (y no se reconoce como fecha):**
Crea una columna auxiliar `release_clean` y usa:
```gs
=SUBSTITUTE(SUBSTITUTE(D2, "-", "/"), ".", "/")
```
Luego crea `release_iso` con:
```gs
=TEXT(DATEVALUE(E2), "yyyy-mm-dd")
```

**Explicación rápida:**
- `SUBSTITUTE` cambia separadores para dejar todo uniforme.
- `DATEVALUE` intenta convertir texto → fecha.
- `TEXT(...,"yyyy-mm-dd")` imprime la fecha en formato ISO.

> Si aparece un error, no lo escondas: identifica el valor original y corrígelo con Buscar/Reemplazar.

---

### 1.3 Booleanos como TRUE/FALSE (`explicit_bool`)
**Problema:** valores como `Yes`, `no`, `TRUE`, `0/1`.

Crea una columna `explicit_bool` y usa:
```gs
=OR(LOWER(J2)="yes", LOWER(J2)="true", J2=1)
```

**Qué hace:**
- `LOWER(J2)` convierte el texto a minúsculas para comparar sin preocuparte por mayúsculas.
- `OR(...)` devuelve `TRUE` si **alguna** condición se cumple.

---

### 1.4 Números limpios (`streams_num`, `chart_peak_num`)

#### `streams_num`
**Problema:** `streams` puede venir como texto con comas, guiones bajos o espacios (por ejemplo: `1,800,000,000` o `1_000_000_000`).

Crea `streams_num` con:
```gs
=VALUE(SUBSTITUTE(SUBSTITUTE(SUBSTITUTE(G2, ",", ""), "_", ""), " ", ""))
```

**Qué hace:**
- Cada `SUBSTITUTE(..., "", ...)` elimina un separador.
- `VALUE(...)` convierte texto → número.

#### `chart_peak_num`
Crea `chart_peak_num` con:
```gs
=IF(K2="",,VALUE(K2))
```

**Qué hace:**
- `IF(K2="",, ...)` deja la celda vacía si no hay dato.
- `VALUE(K2)` convierte el valor a número.

> Si `VALUE(K2)` da error, revisa el contenido original (a veces hay letras o símbolos) y límpialo con Buscar/Reemplazar.

**Recuerda:** registra en `change_log` que agregaste columnas de normalización y qué decisión tomaste.


---
## 2) Duración uniforme y *split* de autores

### 2.1 Duración a segundos (`duration_sec`)
**Meta:** llevar todo a **segundos** para poder promediar y comparar.

**Paso 1 — Si tienes valores tipo `3m51s`, conviértelos a `3:51` con Buscar y reemplazar (Ctrl+H)**
- Buscar `m` → Reemplazar con `:`
- Buscar `s` → Reemplazar con *(vacío)*

**Paso 2 — Crea la columna `duration_sec`**
En la fila 2 usa:
```gs
=IF(
  N2="",
  ,
  IF(
    ISNUMBER(FIND(":", N2)),
    VALUE(LEFT(N2, FIND(":", N2)-1))*60 + VALUE(RIGHT(N2, LEN(N2)-FIND(":", N2))),
    VALUE(N2)
  )
)
```

**Explicación por partes:**
- `FIND(":", N2)` busca la posición de `:`.
- `LEFT(...)` toma los **minutos** (antes de `:`).
- `RIGHT(...)` toma los **segundos** (después de `:`).
- `LEN(N2)` ayuda a calcular cuántos caracteres tiene el texto.
- `VALUE(...)` convierte texto → número.
- La estructura `IF(..., ..., ...)` elige entre:
  - formato `mm:ss` → `min*60 + seg`
  - formato numérico → ya son segundos

---

### 2.2 Separar autores (`writers`) en columnas

**Objetivo:** convertir una lista como `Taylor Swift, Max Martin, Shellback` en columnas separadas.

#### Paso 1 — Inserta columnas para autores
A la derecha de `writers`, inserta 3 columnas: `writer_1`, `writer_2`, `writer_3`.

#### Paso 2 — Divide por coma con `SPLIT`
En `writer_1` (fila 2):
```gs
=SPLIT(TRIM(H2), ",")
```

**Qué hace:**
- `TRIM(H2)` limpia espacios al inicio/fin.
- `SPLIT(...,",")` separa el texto por comas y reparte en columnas.

#### Paso 3 — Limpia cada nombre (capitalización y espacios)
Crea columnas `writer_1_clean`, `writer_2_clean`, `writer_3_clean`.

Ejemplo para `writer_1_clean` (si `writer_1` quedó en M):
```gs
=IF(M2="",,PROPER(TRIM(M2)))
```

**Qué hace:**
- Si está vacío, deja vacío.
- Si tiene texto, aplica `TRIM` + `PROPER`.

**Tip:** si ves nombres con espacios dobles en el medio, aplica Buscar/Reemplazar sobre esas columnas: `␠␠` → `␠`.

> Para esta clase, con esto es suficiente. Conteos avanzados por autor los haremos más adelante con tablas dinámicas.


---
## 3) Duplicados y llaves de unicidad

### 3.1 Clave canónica (`key`)
**Objetivo:** detectar duplicados reales combinando `song + album_norm + track_no`.

Crea una columna `key` y usa (fila 2):
```gs
=LOWER(TRIM(A2)) & "||" & LOWER(TRIM(L2)) & "||" & TEXT(C2, "00")
```

**Explicación:**
- `TRIM` limpia espacios al inicio/fin.
- `LOWER` pasa todo a minúsculas para comparar sin problemas.
- `TEXT(C2,"00")` fuerza dos dígitos para el número de pista (`01`, `09`, `10`).
- `||` es solo un separador para que la clave sea legible.

> Si notas que el campo `song` trae espacios raros dentro del texto, primero usa Buscar/Reemplazar (doble espacio → espacio).

### 3.2 Bandera de duplicado (`dupe_flag`)
Crea `dupe_flag` con:
```gs
=COUNTIF($Q$2:$Q, $Q2) > 1
```

**Qué hace:** cuenta cuántas veces aparece la misma `key`. Si aparece más de una vez, marca `TRUE`.

### 3.3 Resolver duplicados (forma simple y segura)
Para principiantes, la forma más segura es:
1. Filtra `dupe_flag = TRUE`.
2. Decide qué fila conservar (por ejemplo, la que tenga más `streams_num` o la que esté más completa).
3. Registra la decisión en `change_log`.

**Opcional (herramienta de Sheets):**
- Ve a **Datos → Limpieza de datos → Quitar duplicados**.
- Asegúrate de usar la columna `key` como criterio.

> Nota: solo uses “Quitar duplicados” cuando estés seguro de que la `key` está correcta.


---
## 4) KPIs y Resumen (con y sin Tabla Dinámica)

### 4.1 Sin Pivot — Totales y promedios por álbum
Crea una pestaña `summary_albums`.

**Lista de álbumes únicos (A2):**
```gs
=SORT(UNIQUE(clean_songs!L2:L))
```

**Qué hace:**
- `UNIQUE` saca los valores sin repetir.
- `SORT` los ordena.

**Suma de streams por álbum (B2):**
```gs
=IF($A2="","",SUMIF(clean_songs!L:L, $A2, clean_songs!H:H))
```

**Promedio de duración (seg) por álbum (C2):**
```gs
=IF($A2="","",AVERAGEIF(clean_songs!L:L, $A2, clean_songs!O:O))
```

**Streams por minuto (D2):**
```gs
=IF($C2="","", $B2 / ($C2/60))
```

> Importante: si en tu archivo las columnas no coinciden (por ejemplo streams_num no está en H), ajusta las letras de columna.

---

### 4.2 Con Tabla Dinámica (Pivot)
**Insertar → Tabla dinámica** usando `clean_songs`:
- **Filas:** `album_norm`
- **Valores:** `streams_num` (Suma), `duration_sec` (Promedio)

**Ventaja:** rápido.  
**Desventaja:** es menos transparente que ver las fórmulas en celdas.

---

### 4.3 Top 10 canciones por streams (sin Pivot, sin QUERY)
En una pestaña `top_songs`:
1. Copia los encabezados que necesitas (por ejemplo: `song`, `album_norm`, `streams_num`).
2. Usa **Datos → Ordenar rango** por `streams_num` de mayor a menor.
3. Quédate con las primeras 10 filas.

> En esta sesión evitamos `QUERY` para mantenerlo accesible a quien nunca ha usado Sheets.


---
## 5) Gráficos y checklist final

### Gráficos (rápidos y claros)
1. **Columnas:** `streams_sum` por `album_norm` (desde `summary_albums`).
2. **Histograma:** distribución de `duration_sec` (en `clean_songs`).

**Formato recomendado:**
- `streams_num` → número con separador de miles.
- `duration_sec` → número entero.
- Si conviertes a minutos (`duration_sec/60`) → 1–2 decimales.

### Checklist final
- `change_log` completo y claro (qué, dónde, por qué).
- `clean_songs` sin duplicados (o duplicados documentados) y con tipos correctos.
- `summary_albums` coherente con los datos.
- Archivo compartido con el instructor.


---
## 6) Resumen de aprendizaje (takeaways)
- Normalizar **texto** con `TRIM`, `LOWER`, `PROPER`.
- Corregir variantes rápidas con **Buscar y reemplazar (Ctrl+H)**.
- Homologar **fechas** con formato y, si hace falta, `SUBSTITUTE` + `DATEVALUE` + `TEXT`.
- Limpiar **números** heterogéneos con `SUBSTITUTE` + `VALUE`.
- Convertir **duración** a segundos con `FIND` + `LEFT/RIGHT/LEN`.
- Detectar **duplicados** con una clave (`key`) y `COUNTIF`.
- Construir KPIs con `UNIQUE`, `SUMIF`, `AVERAGEIF`.

---

## 7) Mini evaluación (5 ítems)

**1) Verdadero/Falso:**  
> `TRIM` elimina todos los espacios dentro del texto, no solo los iniciales/finales.  
**Respuesta esperada:** *Falso*. `TRIM` quita espacios al inicio/fin y reduce espacios repetidos, pero no “elimina” palabras ni reemplaza separadores.

**2) Verdadero/Falso:**  
> Si seleccionas una sola columna antes de abrir Buscar y reemplazar (Ctrl+H), puedes aplicar cambios solo a esa columna.  
**Respuesta esperada:** *Verdadero*.

**3) Opción múltiple:** ¿Cuál fórmula es más adecuada para normalizar un texto antes de comparar valores?
- A) `PROPER(A2)`
- B) `LOWER(TRIM(A2))`
- C) `TEXT(A2,"yyyy-mm-dd")`
- D) `VALUE(A2)`

**Respuesta esperada:** *B*.

**4) Práctica breve:**  
En `clean_songs`, crea `release_iso` en formato `yyyy-mm-dd` usando una columna auxiliar `release_clean` (cambiando `-` y `.` por `/`).

**5) Práctica breve:**  
En `clean_songs`, crea `duration_sec` a partir de valores `mm:ss` y/o valores numéricos.

**Rúbrica rápida (auto-chequeo):**
- (1 pt) Normalizaste álbumes con `TRIM` + `PROPER` y corregiste 2–5 variantes con Ctrl+H.
- (1 pt) Fechas ISO correctas.
- (1 pt) Booleanos y numéricos consistentes.
- (1 pt) Duración en segundos.
- (1 pt) Duplicados detectados con `key` y `dupe_flag`.


---
## Rúbrica sugerida (evaluación formativa)
- Detección/corrección de calidad (duplicados, faltantes, consistencia) — **30%**  
- Transformaciones de columna (texto/fecha/número/listas) — **25%**  
- Flujo claro y `change_log` completo — **15%**  
- Estadísticos resumen correctos y tratamiento de nulos — **15%**  
- Gráfico claro + narrativa — **10%**  
- Uso responsable de IA documentado — **5%**

## Cierre
- Primero **limpiar**, luego **analizar** y finalmente **comunicar**.  
- Las reglas explícitas y la trazabilidad inspiran confianza.  
- Un gráfico simple y una narrativa concisa son más efectivos que complejos sin contexto.

## Siguientes Pasos
- **Próxima sesión:** profundizaremos en el ecosistema de datos y en el flujo de punta a punta.
- **Participación continua:** asistir a Co-Learning y a Sprint Focus, y usar los canales de Discord para hacer preguntas.
- **Recordatorios:** la grabación y recursos utilizados se comparten al finalizar la sesión; en caso de necesitar apoyo adicional, agenda un 1:1.
